# 路徑與檔案系統 | Paths and File Systems

## 📝 詳解範例 | Worked Examples

---

## 💡 本檔案目的

本檔案提供 **4 個循序漸進的詳解範例**，每個範例包含：
1. **問題描述**：實際應用情境
2. **分析思路**：如何拆解問題
3. **逐步實作**：程式碼 + 註解
4. **執行結果**：預期輸出
5. **知識點總結**：學到什麼

---

## 範例 1：使用pathlib建立和管理路徑

### 📋 問題描述
展示Path物件的基本操作

難度：基礎

### 🔍 分析思路
1. （根據題目拆解步驟）
2. （思考需要哪些技術）
3. （設計資料結構）

### 💻 逐步實作

In [ ]:
# 範例 1 程式碼
# TODO: 根據需求實作完整程式碼

print("範例 1：使用pathlib建立和管理路徑")
# 實作內容...

### 📚 知識點總結
- ✅ （知識點 1）
- ✅ （知識點 2）
- ✅ （知識點 3）

---

## 範例 2：檔案系統資訊獲取

### 📋 問題描述
取得檔案大小、修改時間、是否存在等資訊

難度：中級

### 🔍 分析思路
1. （根據題目拆解步驟）
2. （思考需要哪些技術）
3. （設計資料結構）

### 💻 逐步實作

In [ ]:
# 範例 2 程式碼
# TODO: 根據需求實作完整程式碼

print("範例 2：檔案系統資訊獲取")
# 實作內容...

### 📚 知識點總結
- ✅ （知識點 1）
- ✅ （知識點 2）
- ✅ （知識點 3）

---

## 範例 3：遞迴遍歷目錄

### 📋 問題描述
搜尋目錄下所有特定副檔名的檔案

難度：中級

### 🔍 分析思路
1. （根據題目拆解步驟）
2. （思考需要哪些技術）
3. （設計資料結構）

### 💻 逐步實作

In [ ]:
# 範例 3 程式碼
# TODO: 根據需求實作完整程式碼

print("範例 3：遞迴遍歷目錄")
# 實作內容...

### 📚 知識點總結
- ✅ （知識點 1）
- ✅ （知識點 2）
- ✅ （知識點 3）

---

## 範例 4：跨平台路徑處理

### 📋 問題描述
處理Windows和Linux路徑差異

難度：進階

### 🔍 分析思路
1. （根據題目拆解步驟）
2. （思考需要哪些技術）
3. （設計資料結構）

### 💻 逐步實作

In [ ]:
# 範例 4 程式碼
# TODO: 根據需求實作完整程式碼

print("範例 4：跨平台路徑處理")
# 實作內容...

### 📚 知識點總結
- ✅ （知識點 1）
- ✅ （知識點 2）
- ✅ （知識點 3）

---